# Average NL intensity
In the notebook, we'll try to compute the average NL intensity in Africa for 2015.

In [4]:
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
from neal_utils import *
import glob
from osgeo import gdal, osr

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Testing

In [ ]:
A = Image.open('/atlas/u/nj/viirs/2015/1/2/SVDNB_npp_20150101-20150131_75N060W_vcmcfg_v10_c201505111709.avg_rade9.tif')
A_count = Image.open('/atlas/u/nj/viirs/2015/1/2/SVDNB_npp_20150101-20150131_75N060W_vcmcfg_v10_c201505111709.cf_cvg.tif')
B = Image.open('/atlas/u/nj/viirs/2015/2/2/SVDNB_npp_20150201-20150228_75N060W_vcmcfg_v10_c201504281504.avg_rade9.tif')
B_count = Image.open('/atlas/u/nj/viirs/2015/2/2/SVDNB_npp_20150201-20150228_75N060W_vcmcfg_v10_c201504281504.cf_cvg.tif')

In [ ]:
print A.size
print A_count.size
print B.size
print B_count.size

In [ ]:
t0 = time.time()
A = np.array(A)
A_count = np.array(A_count)
B = np.array(B)
B_count = np.array(B_count)
t1 = time.time()
print 'Converted to arrays: {} seconds'.format(t1-t0)
print A.shape
print A_count.shape
print B.shape
print B_count.shape

In [ ]:
C = np.zeros((18000, 28800, 2))
C_count = np.zeros((18000, 28800, 2))

In [ ]:
t0 = time.time()
C[:,:,0] = A
C[:,:,1] = B
C_count[:,:,0] = A_count
C_count[:,:,1] = B_count
t1 = time.time()
print 'Filling 3D arrays: {} seconds'.format(t1-t0)

In [ ]:
# Using Rachel's code to combine them
t0 = time.time()
D, D_count = get_year_avg(C, C_count)
t1 = time.time()
print 'Averaged observations: {} seconds'.format(t1-t0)

In [ ]:
np.save('/atlas/u/nj/viirs/test.npy', D)
np.save('/atlas/u/nj/viirs/test_counts.npy', D_count)

### Averaging VIIRS data
Now that we know how to do what we want to do, let's write some code to go through the directories and average the observations for all 6 tiles over all 12 months. We'll average two months at a time since we know that works.

In [ ]:
data_dir = '/atlas/u/nj/viirs/2015/'

In [ ]:
t0 = time.time()
for tile in xrange(3,7):
    print 'Starting tile {}: {} seconds'.format(tile, time.time() - t0)
    for month in xrange(1, 13):
        print '  Starting month {}: {} seconds'.format(month, time.time() - t0)
        # If January, set running averages and counts to January data
        if month == 1:
            for fn in glob.glob(data_dir + str(month) + '/' + str(tile) + '/*'):
                if fn[-13:] == 'avg_rade9.tif':
                    avg_obs = Image.open(fn)
                    avg_obs = np.array(avg_obs)
                elif fn[-10:] == 'cf_cvg.tif':
                    total_counts = Image.open(fn)
                    total_counts = np.array(total_counts)
            print '    Loaded data for month {}: {} seconds'.format(month, time.time() - t0)
            rows, cols = avg_obs.shape
            print '      Rows: {}'.format(rows)
            print '      Cols: {}'.format(cols)
        # If any other month, average with previous observations and counts
        else:
            for fn in glob.glob(data_dir + str(month) + '/' + str(tile) + '/*'):
                if fn[-13:] == 'avg_rade9.tif':
                    month_obs = Image.open(fn)
                    month_obs = np.array(month_obs)
                elif fn[-10:] == 'cf_cvg.tif':
                    month_counts = Image.open(fn)
                    month_counts = np.array(month_counts)
            print '    Loaded data for month {}: {} seconds'.format(month, time.time() - t0)
            temp_obs = np.zeros((rows, cols, 2))
            temp_counts = np.zeros((rows, cols, 2))
            temp_obs[:,:,0] = avg_obs
            temp_counts[:,:,0] = total_counts
            temp_obs[:,:,1] = month_obs
            temp_counts[:,:,1] = month_counts
            avg_obs, total_counts = get_year_avg(temp_obs, temp_counts)
            print '    Averaged observations for month {}: {} seconds'.format(month, time.time() - t0)
    obs_fn = data_dir + 'tile' + str(tile) + '_obs.npy'
    counts_fn = data_dir + 'tile' + str(tile) + '_counts.npy'
    np.save(obs_fn, avg_obs)
    print '  Saved observations for tile {}: {} seconds'.format(tile, time.time() - t0)
    np.save(counts_fn, total_counts)
    print '  Saved counts for tile {}: {} seconds'.format(tile, time.time() - t0)

## Filter out points not in Africa
In order to decide how many classes of nightlights we want to have, we want to filter out all the points that aren't in Africa and plot a histogram.

#### Tile 2
Let's start with tile 2.

In [26]:
# Load geotif
#geotif_addr = '/atlas/u/nj/viirs/2015/12/5/SVDNB_npp_20151201-20151231_00N060W_vcmcfg_v10_c201601251413.avg_rade9.tif'
geotif_addr = '/atlas/u/nj/viirs/2015/12/2/SVDNB_npp_20151201-20151231_75N060W_vcmcfg_v10_c201601251413.avg_rade9.tif'
geotif = Image.open(geotif_addr)
# Load averaged observations
tile_obs_fn = '/atlas/u/nj/viirs/2015/data/tile2_obs.npy'
tile_obs = np.load(tile_obs_fn)
rows, cols = tile_obs.shape
print 'Rows: {}'.format(rows)
print 'Columns: {}'.format(cols)

Rows: 18000
Columns: 28800


In [12]:
# Produce list of pixel pairs
low_x = 0
high_x = 1000
low_y = 0
high_y = 1000
pixel_pairs = make_pixel_pairs(low_x, high_x, low_y, high_y)
print len(pixel_pairs)

In [13]:
# Convert pixel pairs to lat/lon pairs
t0 = time.time()
lat_lon_pairs = pixel_to_lat_lon(geotif_addr, pixel_pairs)
print 'Took {} seconds'.format(time.time() - t0)

Took 2.08362388611 seconds


In [38]:
# Convert lat/lon pairs to arrays
lats, lons = convert_latlon_pairs_to_arrays(lat_lon_pairs)
within = check_batch_within_africa(lats, lons)
t1 = time.time()
print 'Checked {} points in {} seconds'.format(within.size, t1-t0)

Checked 10000 points: 0.379335165024 seconds
Checked 20000 points: 0.759891986847 seconds
Checked 30000 points: 1.13965797424 seconds
Checked 40000 points: 1.51730799675 seconds
Checked 50000 points: 1.89725112915 seconds
Checked 60000 points: 2.27466607094 seconds
Checked 70000 points: 2.65358114243 seconds
Checked 80000 points: 3.03345704079 seconds
Checked 90000 points: 3.41781997681 seconds
Checked 100000 points: 3.80042695999 seconds
Checked 110000 points: 4.18270206451 seconds
Checked 120000 points: 4.56223297119 seconds
Checked 130000 points: 4.94053602219 seconds
Checked 140000 points: 5.31884598732 seconds
Checked 150000 points: 5.70122003555 seconds
Checked 160000 points: 6.08313298225 seconds
Checked 170000 points: 6.46814608574 seconds
Checked 180000 points: 6.85870909691 seconds
Checked 190000 points: 7.24065613747 seconds
Checked 200000 points: 7.62395906448 seconds
Checked 210000 points: 8.04642605782 seconds
Checked 220000 points: 8.42741107941 seconds
Checked 230000 po

In [40]:
# Get nl values for pixels in Africa
pixel_pairs_in_africa = np.array(pixel_pairs)
nl_values = np.zeros((pixel_pairs_in_africa.shape[0],))
t0 = time.time()
for i in xrange(pixel_pairs_in_africa.shape[0]):
    nl_values[i] = tile_obs[pixel_pairs_in_africa[i,1], pixel_pairs_in_africa[i,0]]
    if i % 1000 == 0:
        print 'Finished {} points: {} seconds'.format(i, time.time() - t0)

Finished 0 points: 0.000334024429321 seconds
Finished 1000 points: 0.00144600868225 seconds
Finished 2000 points: 0.0023500919342 seconds
Finished 3000 points: 0.00324106216431 seconds
Finished 4000 points: 0.00415706634521 seconds
Finished 5000 points: 0.00505089759827 seconds
Finished 6000 points: 0.00603890419006 seconds
Finished 7000 points: 0.00725102424622 seconds
Finished 8000 points: 0.00841307640076 seconds
Finished 9000 points: 0.00954604148865 seconds
Finished 10000 points: 0.0107159614563 seconds
Finished 11000 points: 0.0116329193115 seconds
Finished 12000 points: 0.0125350952148 seconds
Finished 13000 points: 0.0134248733521 seconds
Finished 14000 points: 0.0143160820007 seconds
Finished 15000 points: 0.0152819156647 seconds
Finished 16000 points: 0.0161929130554 seconds
Finished 17000 points: 0.0170829296112 seconds
Finished 18000 points: 0.0179738998413 seconds
Finished 19000 points: 0.0188620090485 seconds
Finished 20000 points: 0.0197770595551 seconds
Finished 21000 p

In [41]:
pixel_pairs_in_africa[within].shape

(0, 2)

In [42]:
nl_values

array([ 0.7136881 ,  0.70755659,  0.70322937, ...,  0.59321586,
        0.59358504,  0.6381932 ])

#### Where to search
The NW corner of the Africa bounding box is 38N, 20W, and the SE corner is 36S, 53E. Meanwhile, tile 2 covers 0-75N 60W-60E and tile 5 covers 65S-0 60W-60E. Therefore, since tile 2 is 18000x28800, we should search from rows 8880-18000 and from columns 4800-27120.

Tile 5 is 15600x28800. In tile 5, we only need to search between 8E and 44E, so we need to search rows 0-8640 and columns 16320-24960.

In [27]:
(18000 - 8880) * (27120 - 4800)

203558400

In [28]:
long_list = []
t0 = time.time()
for i in xrange(203558400):
    long_list.append(i)
    if i % 1000000 == 0:
        print i
        print time.time() - t0

0
0.000239133834839
1000000
0.250798225403
2000000
0.506245136261
3000000
0.754440069199
4000000
1.00337910652
5000000
1.25395417213
6000000
1.50226211548
7000000
1.75250005722
8000000
2.00406813622
9000000
2.25257611275
10000000
2.50283718109
11000000
2.7540781498
12000000
3.01169610023
13000000
3.26732420921
14000000
3.51619720459
15000000
3.76623702049
16000000
4.01536917686
17000000
4.26355719566
18000000
4.51077198982
19000000
4.76170516014
20000000
5.01011300087
21000000
5.26011919975
22000000
5.50683307648
23000000
5.7553319931
24000000
6.00355219841
25000000
6.25344204903
26000000
6.50084519386
27000000
6.75167107582
28000000
7.00029921532
29000000
7.24666619301
30000000
7.49535322189
31000000
7.7419500351
32000000
7.98907518387
33000000
8.23817515373
34000000
8.48533511162
35000000
8.73209214211
36000000
8.98018717766
37000000
9.22649717331
38000000
9.47392416
39000000
9.72068619728
40000000
9.96989607811
41000000
10.2165651321
42000000
10.4637510777
43000000
10.7119350433
440

In [29]:
long_array = np.array(long_list)

In [30]:
long_array.shape

(203558400,)

In [31]:
# Pixels to search
lowest_x = 4800
highest_x = 27120
lowest_y = 8880
highest_y = 18000

In [37]:
x_grid = range(lowest_x, highest_x, 1000) + [highest_x]
y_grid = range(lowest_y, highest_y, 1000) + [highest_y]
count = 0
for x_idx in xrange(len(x_grid) - 1):
  for y_idx in xrange(len(y_grid) - 1):
    low_x, high_x = x_grid[x_idx], x_grid[x_idx + 1]
    low_y, high_y = y_grid[y_idx], y_grid[y_idx + 1]
    print [(low_x, high_x), (low_y, high_y)]
    count += 1
    print count

[(4800, 5800), (8880, 9880)]
1
[(4800, 5800), (9880, 10880)]
2
[(4800, 5800), (10880, 11880)]
3
[(4800, 5800), (11880, 12880)]
4
[(4800, 5800), (12880, 13880)]
5
[(4800, 5800), (13880, 14880)]
6
[(4800, 5800), (14880, 15880)]
7
[(4800, 5800), (15880, 16880)]
8
[(4800, 5800), (16880, 17880)]
9
[(4800, 5800), (17880, 18000)]
10
[(5800, 6800), (8880, 9880)]
11
[(5800, 6800), (9880, 10880)]
12
[(5800, 6800), (10880, 11880)]
13
[(5800, 6800), (11880, 12880)]
14
[(5800, 6800), (12880, 13880)]
15
[(5800, 6800), (13880, 14880)]
16
[(5800, 6800), (14880, 15880)]
17
[(5800, 6800), (15880, 16880)]
18
[(5800, 6800), (16880, 17880)]
19
[(5800, 6800), (17880, 18000)]
20
[(6800, 7800), (8880, 9880)]
21
[(6800, 7800), (9880, 10880)]
22
[(6800, 7800), (10880, 11880)]
23
[(6800, 7800), (11880, 12880)]
24
[(6800, 7800), (12880, 13880)]
25
[(6800, 7800), (13880, 14880)]
26
[(6800, 7800), (14880, 15880)]
27
[(6800, 7800), (15880, 16880)]
28
[(6800, 7800), (16880, 17880)]
29
[(6800, 7800), (17880, 18000)]
3

In [43]:
list1 = []
list2 = [9, 3, 5]

In [44]:
list1

[]

In [45]:
list2

[9, 3, 5]

In [46]:
list1 += list2

In [47]:
list1

[9, 3, 5]